### Carlos Morán y Carlos Tardón grupo 9

In [25]:
from search import *

In [33]:
## grados.py
import csv
import sys

# diccionario de nombres de personas con ids 
names = {}
# diccionario: name, birth, movies (conjunto de movie_ids)
people = {}
# movie_ids to a dictionary of: title, year, stars (a set of person_ids)
movies = {}

def load_data(directory):
    """
    Load data from CSV files into memory.
    """
    # Cargamos el archivo people
    with open(f"{directory}/people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            

    # cargamos el archivo movies
    with open(f"{directory}/movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # cargamos el archivo stars
    with open(f"{directory}/stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass


def main(directory="large"):
    # Load data from files into memory
    print("Cargando los datos...")
    load_data(directory)
    print("Datos cargados.")
    source = person_id_for_name(input("Nombre: "))
    if source is None:
        sys.exit("Esa persona no se encuentra.")
    target = person_id_for_name(input("Nombre: "))
    if target is None:
        sys.exit("Esa persona no se encuentra.")

    path = shortest_path(source, target)

    if path is None:
        print("No están conectados.")
    else:
        degrees = len(path)
        print(f"{degrees} grados de separacion.")
        path = [(None, source)] + path
        for i in range(degrees):
            person1 = people[path[i][1]]["name"]
            person2 = people[path[i + 1][1]]["name"]
            movie = movies[path[i + 1][0]]["title"]
            print(f"{i + 1}: {person1} y {person2} participaron en {movie}")

def person_id_for_name(name):
    """
    Returns the IMDB id for a person's name,
    resolving ambiguities as needed.
    """
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]


def neighbors_for_person(person_id):
    """
    Returns (movie_id, person_id) pairs for people who starred with a given person.
    """
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
        for person_id in movies[movie_id]["stars"]:
            neighbors.add((movie_id, person_id))
    return neighbors


#if __name__ == "__main__":
 #   main()


#### Decisiones
El estado va a estar representado por el nombre de la persona que estamos considerando actualmente. Por tanto, el estado final se corresponde con la persona que buscamos, y el inicial con la persona desde la que partimos.

La representación de acciones viene ya dada, y es una tupla (movie_id, person_id) que indica la película en la que actúa el actor (estado actual), y la siguiente persona (que también actúa en esa película), y que por tanto será el próximo estado.  
Nos interesa llegar al estado final dando el mínimo número de saltos, por lo que la búsqueda adecuada es una búsqueda en anchura.  
En este problema no consideramos el uso de una heurística, pues inicialmente no tenemos información de la estructura del grafo. Tal vez se podría preprocesar, pero no hemos indagado en esa opción, e intuimos que sería inviable para grafos dispersos con muchos nodos.

In [28]:
class Degrees(Problem):

    def __init__(self, initial, goal):
        self.initial = initial
        self.goal = goal

    def actions(self,estado):
        return neighbors_for_person(estado)

    def result(self,estado,accion):
        return accion[1]

In [29]:

def shortest_path(source, target):
    problem = Degrees(source, target)
    node = breadth_first_graph_search(problem)
    if node == None:
        return node
    return node.solution()


In [6]:
#cargamos los datos
load_data("small")

In [16]:
load_data("large") 

In [8]:
name="Emma Watson"
person_id=person_id_for_name(name)

In [8]:
print(person_id_for_name("Emma Watson"))

914612


In [9]:
neighbors_for_person(person_id)

{('1201607', '2091'),
 ('1201607', '342488'),
 ('1201607', '705356'),
 ('1201607', '914612'),
 ('1659337', '3009232'),
 ('1659337', '503567'),
 ('1659337', '748620'),
 ('1659337', '914612'),
 ('1781796', '2354'),
 ('1781796', '236462'),
 ('1781796', '382268'),
 ('1781796', '447905'),
 ('1781796', '5042'),
 ('1781796', '705356'),
 ('1781796', '914612'),
 ('1959490', '124'),
 ('1959490', '128'),
 ('1959490', '164'),
 ('1959490', '914612'),
 ('2132285', '3441152'),
 ('2132285', '4583512'),
 ('2132285', '4903197'),
 ('2132285', '914612'),
 ('2771200', '1265802'),
 ('2771200', '1405398'),
 ('2771200', '1812656'),
 ('2771200', '914612'),
 ('295297', '1321'),
 ('295297', '342488'),
 ('295297', '705356'),
 ('295297', '914612'),
 ('304141', '341743'),
 ('304141', '342488'),
 ('304141', '705356'),
 ('304141', '914612'),
 ('3281548', '3154303'),
 ('3281548', '6073955'),
 ('3281548', '658'),
 ('3281548', '914612'),
 ('330373', '342488'),
 ('330373', '705356'),
 ('330373', '843059'),
 ('330373', '9

In [34]:
main("large")

Cargando los datos...
Datos cargados.
Nombre: Emma Watson
Nombre: Jennifer Lawrence
3 grados de separacion.
1: Emma Watson y Ethan Hawke participaron en Regression
2: Ethan Hawke y Chris Pratt participaron en The Magnificent Seven
3: Chris Pratt y Jennifer Lawrence participaron en Passengers


#### Caso sin solución  

In [19]:
main("small")

Cargando los datos...
Datos cargados.
Nombre: Emma Watson
Nombre: Kevin Bacon
No están conectados.


### Posibles complicaciones
<ul>
    <li> Podrían pedirnos que, en caso de haber varias soluciones, demos aquella cuyas películas sean más antiguas. En ese caso habría que modificar la función neighbors_for_person, para que la lista de acciones esté ordenada de menor a mayor año, y así la búsqueda en anchura siga ese orden concreto. Otra opción sería ordenar las películas de cada actor después de haberlas cargado, pero sería menos eficiente para soluciones cortas, pues implicaría ordenar todas las películas de todos los actores, mientras que la primera solución solo obliga a ordenar una vez se llega a un actor concreto. </li>
    <li> Podrían decirnos que solo podemos considerar películas de un siglo dado. En ese caso, nuevamente modificaríamos la función neighbors_for_person para que solo devuelva acciones válidas.</li>
    <li> Podrían pedirnos la pareja de actores que tienen mayor grado de separación entre sí. En ese caso, habría que cambiar de algoritmo, ejecutar Floyd sobre todo el grafo (que calcula los caminos mínimos entre cualesquiera dos nodos), y quedarnos con la pareja (o parejas) con mayor grado de separación.</li>
</ul>
